In [9]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import scipy.optimize as opti
from scipy.optimize import LinearConstraint, Bounds
import wrds
import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Exercise 2

In [2]:
db = wrds.Connection()
#antb95
#goqhuB-1hafqe-dojvix

Enter your WRDS username [bedanian]:antb95
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [3]:
#import data
names = db.get_table(library='crsp', table='stocknames')
names =names.loc[:,['permco' , 'ticker']]
Ticker = 'MSFT'
inter = names['permco'].where(names['ticker'] == Ticker)
inter.dropna(inplace = True)
Permco = inter.iloc[0]
df = db.raw_sql("select prc, date from crsp.dsf where permco in ("+str(Permco)+") and date >='2011-04-07' and date <='2016-04-07'")
df.columns = ['MSFT' , 'Date']
df.set_index('Date',inplace = True)

In [4]:
df2 = pd.DataFrame()
df2 = (np.log(df) - np.log(df).shift(1))
df2.dropna(inplace = True)
df2.columns = ['R']
df2['Z'] = np.random.normal(0,1,len(df2))
df2.reset_index(inplace = True)

In [5]:
def sigma_t(alpha,x,t,sigma_0):
    if t == 0:
        return sigma_0
    else:
        return np.sqrt(alpha[0]+alpha[1]*x[t-1]**2+alpha[2]*sigma_t(alpha,x,t-1,sigma_0)**2)

In [6]:
def L(alpha,x,sigma_0):
    l = 0
    for k in range(len(x)):
        sigma = sigma_t(alpha,x,k,sigma_0)
        l *= norm.pdf(x[k]/sigma)/sigma
    return l

In [10]:
index_0 = df2[df2['Date'] == datetime.date(2013,4,8)].index.values[0]
i = 0 
x0 = [(0.2**2)/(1-0.1-0.8),0.1,0.8]
bnds = Bounds([10**(-8),0,0], [np.inf,np.inf,np.inf], keep_feasible=True)
x = list(df2['R'])
while i+index_0 < len(x):  
    xi = x[i:index_0+i]
    mu = np.mean(xi)
    xi = xi - mu
    sigma_0 = np.std(xi)
    fun = lambda alpha: -L(alpha,xi,sigma_0)
    print(opti.minimize(fun, x0, method='SLSQP', bounds = bnds).x)
    i=i+1
    if i > 20:
        break  

[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]
[0.4 0.1 0.8]


In [11]:
df2

,Date,R,Z
0,2011-04-08,-0.004974,0.413620
1,2011-04-11,-0.003458,-0.032286
2,2011-04-12,-0.013173,-1.128588
3,2011-04-13,-0.000390,-1.152169
4,2011-04-14,-0.008227,-1.353410
5,2011-04-15,-0.001969,-0.296673
6,2011-04-18,-0.011497,1.407170
7,2011-04-19,0.002787,-0.182258
8,2011-04-20,0.023965,-0.888440
9,2011-04-21,-0.009360,-0.164777
